In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st


In [2]:
# read in open baltimore 2012-2023 dataset
df_raw = pd.read_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data.csv")
df_raw.head()

C:\Users\johbr\AppData\Local\Temp\ipykernel_12532\1278201283.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data.csv")


,OBJECTID,CCNO,CrimeDateTime,CrimeCode,Location,Description,Inside_Outside,Weapon,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,Total_Incidents
0,1,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,NaN,PERSONAL_WEAPONS,833.0,F,15.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.273302,-76.692439,"(39.27330200992213,-76.69243902745305)",NaN,1
1,2,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,NaN,PERSONAL_WEAPONS,833.0,F,15.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.273302,-76.692439,"(39.27330200992213,-76.69243902745305)",NaN,1
2,3,23F08231,2023/06/24 04:01:00+00,4B,600 LUCIA AVE,AGG. ASSAULT,NaN,PERSONAL_WEAPONS,833.0,F,27.0,BLACK_OR_AFRICAN_AMERICAN,NOT_HISPANIC_OR_LATINO,SOUTHWEST,YALE HEIGHTS,39.273302,-76.692439,"(39.27330200992213,-76.69243902745305)",NaN,1
3,4,23F08231,2023/06/24 04:01:00+00,3JK,600 LUCIA AVE,ROBBERY,NaN,PERSONAL_WEAPONS,833.0,M,25.0,BLACK_OR_AFRICAN_AMERICAN,UNKNOWN,SOUTHWEST,YALE HEIGHTS,39.273302,-76.692439,"(39.27330200992213,-76.69243902745305)",NaN,1
4,5,23F08235,2023/06/24 03:45:00+00,5A,3200 LILY AVE,BURGLARY,NaN,NaN,922.0,M,48.0,NaN,HISPANIC_OR_LATINO,SOUTHERN,CHERRY HILL,39.246432,-76.636819,"(39.24643210111462,-76.63681903810716)",NaN,1


In [3]:
print(df_raw.iloc[:, 11])
"""
DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2023 = pd.read_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data.csv")

so, mixed values in 'race' column? doesn't seem right. is there a random float in there?
what's the data type error? if null, why not in "inside/outside"?...
...something weird is going on

Should be fixed by replacing csv read in:
  df_raw = pd.read_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data.csv", dtype={"Race": str})
But i wanna poke it first
"""

0         BLACK_OR_AFRICAN_AMERICAN
1         BLACK_OR_AFRICAN_AMERICAN
2         BLACK_OR_AFRICAN_AMERICAN
3         BLACK_OR_AFRICAN_AMERICAN
4                               NaN
                    ...            
568609    BLACK_OR_AFRICAN_AMERICAN
568610                      UNKNOWN
568611    BLACK_OR_AFRICAN_AMERICAN
568612    BLACK_OR_AFRICAN_AMERICAN
568613    BLACK_OR_AFRICAN_AMERICAN
Name: Race, Length: 568614, dtype: object


'\nDtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.\n  df_2023 = pd.read_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data.csv")\n\nso, mixed values in \'race\' column? doesn\'t seem right. is there a random float in there?\nwhat\'s the data type error? if null, why not in "inside/outside"?...\n...something weird is going on\n\nShould be fixed by replacing csv read in:\n  df_raw = pd.read_csv("Resources/2012-2023_BPD_Victim_Based_Crime_Data.csv", dtype={"Race": str})\nBut i wanna poke it first\n'

In [4]:
# clean 2023 data

# create copy of raw 2012-2023 data
df = df_raw

# split crime date and time
df[["CrimeDate", "CrimeTime"]] = df["CrimeDateTime"].str.split(" ", 1, expand=True)
# delete useless columns
del df['CCNO']
del df['OBJECTID']
del df["CrimeDateTime"]
df.rename(columns={'Inside_Outside': 'Inside/Outside'}, inplace=True)
# insert date and time into beginning of df
df.insert(0, "CrimeDate", df.pop("CrimeDate"))
df.insert(1, "CrimeTime", df.pop("CrimeTime"))

df.CrimeDate = pd.to_datetime(df.CrimeDate, errors='coerce')

df = df[df.CrimeDate <= '2021-04-30']
df.head()


C:\Users\johbr\AppData\Local\Temp\ipykernel_12532\3822003992.py:7: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[["CrimeDate", "CrimeTime"]] = df["CrimeDateTime"].str.split(" ", 1, expand=True)


,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside/Outside,Weapon,Post,Gender,Age,Race,Ethnicity,District,Neighborhood,Latitude,Longitude,GeoLocation,Premise,Total_Incidents
83615,2021-04-16,13:30:00+00,6E,1700 N BENTALOU ST,LARCENY,O,NaN,723.0,M,28.0,BLACK_OR_AFRICAN_AMERICAN,NaN,WESTERN,COPPIN HEIGHTS/ASH-CO-EAST,39.307800,-76.653923,"(39.3078,-76.653923)",STREET,1
83616,2021-04-16,10:32:00+00,4D,200 N EUTAW ST,AGG. ASSAULT,I,HANDS,114.0,F,50.0,BLACK_OR_AFRICAN_AMERICAN,NaN,CENTRAL,DOWNTOWN,39.291551,-76.621136,"(39.291551,-76.621136)",MARKET STALLS,1
83617,2021-04-16,15:20:00+00,6J,1000 PATAPSCO ST,LARCENY,NaN,NaN,942.0,F,25.0,WHITE,NaN,SOUTHERN,FEDERAL HILL,39.277654,-76.613765,"(39.277654,-76.613765)",NaN,1
83618,2021-04-16,07:00:00+00,7A,2200 WILKENS AVE,AUTO THEFT,O,NaN,834.0,F,20.0,UNKNOWN,NaN,SOUTHWEST,CARROLLTON RIDGE,39.280495,-76.650988,"(39.280495,-76.650988)",STREET,1
83619,2021-04-16,21:40:00+00,3BJ,1600 EDISON HWY,ROBBERY - CARJACKING,O,NaN,332.0,M,38.0,UNKNOWN,NaN,EASTERN,BEREA,39.309893,-76.573469,"(39.309892500982698,-76.573469434507444)",STREET,1


In [5]:
# write to csv
df.to_csv("Resources/2012-2021_BPD_Victim_Based_Crime_Data_clean.csv")